In [ ]:
'''
Jupyter notebook for exploring data in CSV files
downloaded from National Endowment for the Humanities.

Source: https://securegrants.neh.gov/publicquery/main.aspx

author: Samuel J. Huskey

'''

import pandas as pd
import os
import inspect
pd.options.mode.chained_assignment = None

In [ ]:
'''
I downloaded CSV files for each institution in the 
NEH's database and put them in an "NEH" directory.

Let's get the list of files from the NEH directory.
'''
files = os.listdir('../NEH/AAU')

In [ ]:
# List the files so that we'll know what we have.
for file in files:
    print(file)

In [ ]:
''' 
Count the files. There should be 35, since there are 
35 public AAU universities currently.
'''
len(files)

In [ ]:
'''
Make a big dataframe so that we can use Pandas
to analyze the data.
'''
dfs = {}
for file in files:
    # Strip '.csv' from the name of the file.
    filename = file[:-4]
    filepath = '../NEH/AAU/' + file
    df = pd.read_csv(filepath)
    dfs.update({filename:df})
big_frame = pd.concat([(dfs.get(i)) for i in dfs.keys()])

In [ ]:
# Get a big picture of the data using info().
big_frame.info()

In [ ]:
# Preview the data using head() to show the first five rows.
big_frame.head()

In [ ]:
# Use set() to get a list of institutions included in the data.
set(big_frame['Institution'])

In [ ]:
# We can also use Pandas' unique() method to do the same thing.
big_frame['Institution'].unique()

In [ ]:
'''
Turn that information into a list so that we can
limit the data to AAU institutions in a later step.
'''
institution_list = big_frame['Institution'].unique().tolist()
institution_list

In [ ]:
# Manually make a list of only AAU institutions.
aau_list = ['University of Kansas, Lawrence',
            'Georgia Tech',
            'University of Maryland, College Park',
            'Mizzou',
            'Board of Trustees of the University of Illinois',
            'UCLA; Regents of the University of California, Los Angeles',
            'Penn State',
            'University of Florida',
            'Purdue University',
            'Rutgers, The State University of New Jersey',
            'Michigan State University',
            'University of Wisconsin System',
            'Trustees of Indiana University',
            'Texas A & M University, College Station',
            'Regents of the University of California, Irvine',
            'University of North Carolina at Chapel Hill', 
            'University of Virginia', 
            'University of Minnesota', 
            'University of California, Berkeley',
            'Regents of the University of Colorado, Boulder',
            'University of Texas, Austin',
            'Regents of the University of California, San Diego',
            'University of Washington',
            'Regents of the University of California, Davis',
            'University of Utah',
            'Regents of the University of California, Santa Barbara',
            'SUNY Research Foundation, Stony Brook',
            'Regents of the University of Michigan',
            'Regents of the University of California, Santa Cruz',
            'University of Iowa',
            'SUNY Research Foundation, Buffalo State College',
            'Arizona Board of Regents',
            'Ohio State University',
            'University of Pittsburgh',
            'University of Oregon',
           ]

In [ ]:
# How many institutions are in the aau_list? There should be 35.
len(aau_list)

In [ ]:
# Use isin() to filter the data so that it includes only AAU institutions.
aau_frame = big_frame[big_frame['Institution'].isin(aau_list)]

# Use print() to show the results
print(f"Original: ", len(big_frame))
print(f"Filtered: ", len(aau_frame))

In [ ]:
# Make a dataframe with only the columns that we need.
# First: evaluate the columns available.
aau_frame.columns

In [ ]:
# What's in "ApplicantType"?
aau_frame['ApplicantType']

In [ ]:
'''
That's not very helpful. Let's use head() 
to get a preview of the data.
'''
pd.set_option('display.max_columns', None)
aau_frame.head()

In [ ]:
'''
We'll probably want at least Institution, YearAwarded, 
ProjectTitle, PrimaryDiscipline, ProgramName, 
DivisionName, and AwardOutright.

Let's make a frame with those columns.
'''
data = aau_frame[['Institution','ProjectTitle','PrimaryDiscipline','ProgramName','DivisionName','AwardOutright','YearAwarded']]
data

In [ ]:
# Let's find out more about the awards.
data['ProgramName'].unique().tolist()

In [ ]:
'''
Some of those awards don't look relevant to AAU considerations.
Let's look at the divisions that award the grants.
'''
data['DivisionName'].unique().tolist()

In [ ]:
# Let's get counts of the grants from those programs.
data['DivisionName'].value_counts()

In [ ]:
# Bar graph, anyone?
data['DivisionName'].value_counts().plot.barh()

In [ ]:
'''
Okay, let's filter the data to show only
grants from Research Programs, Digital Humanities
and Fellowship and Seminars.
'''
programs = ['Research Programs', 'Digital Humanities','Fellowships and Seminars']
aau_research = data[data['DivisionName'].isin(programs)]
print(f"Original: ", len(big_frame))
print(f"AAU Only: ", len(aau_frame))
print(f"Research Programs Only: ", len(aau_research))

In [ ]:
# Use value_counts() to get the number of awards per institution.
total_number = aau_research['Institution'].value_counts()
total_number

In [ ]:
# Let's see that as a bar graph.
total_number.plot.barh(figsize=(10,10))

In [ ]:
# How about total amount?
total_amount = aau_research.groupby('Institution')['AwardOutright'].sum().sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Use nlargest() to get the top ten institutions by number of grants received.
top_ten = aau_research['Institution'].value_counts().nlargest(10).to_frame()
top_ten

In [ ]:
# Let's make a pie chart of the top ten!
top_ten.plot.pie(y='Institution')

In [ ]:
'''
That's kind of ugly! 
Let's get rid of the label and legend and make it bigger.
'''
top_ten.plot.pie(y='Institution',label='',legend=False,figsize=(10,10),fontsize=14)

In [ ]:
# Let's take a look at dates.
print(f'The earliest date: ', aau_research['YearAwarded'].min())
print(f'The latest date: ', aau_research['YearAwarded'].max())

In [ ]:
# Make dataframes of the earliest and most recent years per institution.
maximum = aau_research.groupby(['Institution'])['YearAwarded'].max().reset_index()
minimum = aau_research.groupby(['Institution'])['YearAwarded'].min().reset_index()

In [ ]:
# Merge the minimum and maximum frames.
merged = pd.merge(minimum,maximum, on='Institution').rename(columns={'YearAwarded_x':'Earliest','YearAwarded_y':'Latest'})
merged

In [ ]:
# How many disciplines are represented in this dataset?
aau_research['PrimaryDiscipline'].nunique()

In [ ]:
# What are the top twenty disciplines in number of grants?
aau_research['PrimaryDiscipline'].value_counts().nlargest(20).to_frame().reset_index()

In [ ]:
aau_research['PrimaryDiscipline'].value_counts().nlargest(20).plot.pie(label=False,fontsize=14,figsize=(10,10))

In [ ]:
'''
Now let's do some textual analysis!
Let's look at the ProjectTitle data.
'''
aau_research['ProjectTitle']

In [ ]:
'''
Hmm. "Tite not available" isn't very useful.
Let's get rid of it.
'''
real_titles = aau_research[aau_research['ProjectTitle'] != 'Title not available']

In [ ]:
'''
From this point forward, I'm drawing heavily on 
https://shorturl.at/ceI39.

Let's turn all of the words into lower case so that
Capital and capital, e.g., are not treated as different words.
'''
real_titles['ProjectTitle_lower'] = real_titles['ProjectTitle'].str.lower()
real_titles['ProjectTitle_lower']

In [ ]:
'''
We'll need to do some natural language processing,
so let's import some stuff from the Natural Language
Toolkit (NLTK).
'''
from nltk.tokenize import RegexpTokenizer
regexp = RegexpTokenizer('\w+')

In [ ]:
'''
Let's tokenize the titles so that we can
do some NLTK operations on individual words.

I'm using apply() to, well, apply the tokenizer to 
each row.
'''
real_titles['TitleToken'] = real_titles['ProjectTitle_lower'].copy(deep=True).apply(regexp.tokenize)
real_titles['TitleToken']

In [ ]:
'''
Let's omit the little words (a.k.a. 'stopwords'), 
like 'the', 'an', 'of', etc.
For that, we'll need a stopwords list.
'''
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words("english")

In [ ]:
'''
I'm using a lambda function to filter out the 
stop words.
'''
real_titles['TitleStop'] = real_titles['TitleToken'].apply(lambda x: [item for item in x if item not in stopwords])
real_titles['TitleStop']

In [ ]:
'''
Now, let's prepare the lists for text mining operations.

First, turn the tokenized titles back into strings.
'''
real_titles['TitleNoShorts'] = real_titles['TitleStop'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))

In [ ]:
# Wordnet has helpful tools for text mining.
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
'''
Use the wordnetlemmatizer to turn the title strings
into lemmata for further processing.
'''
from nltk.stem import WordNetLemmatizer
wordnet_lem = WordNetLemmatizer()

real_titles['TitleLemmatized'] = real_titles['TitleNoShorts'].apply(wordnet_lem.lemmatize)
real_titles['TitleLemmatized']

In [ ]:
'''
Check whether we've lost any words along the way.

Here I'm using is_equal() and value_counts() to verify
that TitleNoShorts and TitleLemmatized are the same size.
'''
real_titles['is_equal']= (real_titles['TitleNoShorts']==real_titles['TitleLemmatized'])
real_titles.is_equal.value_counts()

In [ ]:
# Make a big pile of words!
all_words = ' '.join([word for word in real_titles['TitleLemmatized']])
all_words

In [ ]:
'''
Word cloud anyone? Note that you'll need to install
wordcloud to do this. In a terminal, do 
`conda install -c conda-forge wordcloud`.
'''
%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud

wordcloud = WordCloud(width=600, 
                     height=400, 
                     random_state=2, 
                     max_font_size=100).generate(all_words)

plt.figure(figsize=(10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off');

In [ ]:
# Let's look at frequency of usage.

from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

words = nltk.word_tokenize(all_words)
fd = FreqDist(words)

In [ ]:
# What are the 'n' most common words in titles?
fd.most_common(3)

In [ ]:
# Here are the top ten most frequently used words.
top_10 = fd.most_common(10)
fdist = pd.Series(dict(top_10))
fdist.plot.barh(fontsize=14,figsize=(10,10))

In [ ]:
'''
Let's switch gears and look at how OU
compares to AAU institutions. We'll start
by reminding ourselves what we have 
to work with in the aau_research frame
we made earlier.
'''
aau_research.info()

In [ ]:
# Here's a chart of the history of funding to AAU institutions.
aau_research.groupby('YearAwarded')['AwardOutright'].sum().plot.line()

In [ ]:
# Um, what happened after 2020?
post_2020 = aau_research[aau_research['YearAwarded'] >= 2021 ]
post_2020.groupby('YearAwarded')['AwardOutright'].sum().plot.line()

In [ ]:
# What if we exclude years after 2020?
aau_ante_2021 = aau_research[aau_research['YearAwarded'] <= 2021 ]
aau_ante_2021.groupby('YearAwarded')['AwardOutright'].sum().plot.line()

In [ ]:
'''
Let's start comparing OU to AAU institutions.
First, we need to open oklahoma.csv, which is stored
in a different directory from the AAU files.
'''
oklahoma = pd.read_csv('../NEH/oklahoma.csv')

In [ ]:
# How big is the dataframe?
len(oklahoma)

In [ ]:
'''
It probably has all institutions in Oklahoma.
Let's use unique() to show the unique institutions.
'''
oklahoma['Institution'].unique()

In [ ]:
# Narrow the oklahoma frame to just OU.
ou = oklahoma[oklahoma['Institution'] == 'Board of Regents of the University of Oklahoma']

In [ ]:
len(ou)

In [ ]:
'''
Further filter the frame to NEH programs having
to do with research, like we did with the AAU frame above.
'''
programs = ['Research Programs', 'Digital Humanities','Fellowships and Seminars']
ou_research = ou[ou['DivisionName'].isin(programs)]

In [ ]:
# Show the history of NEH funding at OU
ou_years = ou_research.groupby('YearAwarded')['AwardOutright'].sum().to_frame()
ou_years

In [ ]:
# Here's a dataframe with just the pertinent information about OU.
ou_neh = ou[['Institution','YearAwarded','PDLastname','PDFirstname','ProjectTitle','AwardOutright','PrimaryDiscipline']]
ou_neh

In [ ]:
'''
Which disciplines at OU have been the most successful 
at getting NEH grants?'''
ou_neh['PrimaryDiscipline'].value_counts().nlargest(10).plot.pie(label='',figsize=(10,20),fontsize=14)

In [ ]:
'''
Let's make an OU frame that we can use to compare OU
to AAU institutions.
'''
OU = ou_neh.groupby('YearAwarded')['AwardOutright'].sum().to_frame()

In [ ]:
'''
It will be helpful to have a dictionary of the AAU
institutions. Let's use the list from above to do that.
'''
aau_list = ['University of Kansas, Lawrence',
            'Georgia Tech',
            'University of Maryland, College Park',
            'Mizzou',
            'Board of Trustees of the University of Illinois',
            'UCLA; Regents of the University of California, Los Angeles',
            'Penn State',
            'University of Florida',
            'Purdue University',
            'Rutgers, The State University of New Jersey',
            'Michigan State University',
            'University of Wisconsin System',
            'Trustees of Indiana University',
            'Texas A & M University, College Station',
            'Regents of the University of California, Irvine',
            'University of North Carolina at Chapel Hill', 
            'University of Virginia', 
            'University of Minnesota', 
            'University of California, Berkeley',
            'Regents of the University of Colorado, Boulder',
            'University of Texas, Austin',
            'Regents of the University of California, San Diego',
            'University of Washington',
            'Regents of the University of California, Davis',
            'University of Utah',
            'Regents of the University of California, Santa Barbara',
            'SUNY Research Foundation, Stony Brook',
            'Regents of the University of Michigan',
            'Regents of the University of California, Santa Cruz',
            'University of Iowa',
            'SUNY Research Foundation, Buffalo State College',
            'Arizona Board of Regents',
            'Ohio State University',
            'University of Pittsburgh',
            'University of Oregon',
           ]

aau_dict = {'University of Kansas':'University of Kansas, Lawrence',
            'Georgia Tech':'Georgia Tech',
            'University of Maryland':'University of Maryland, College Park',
            'University of Missouri':'Mizzou',
            'University of Illinois':'Board of Trustees of the University of Illinois',
            'UCLA':'UCLA; Regents of the University of California, Los Angeles',
            'Penn State':'Penn State',
            'University of Florida':'University of Florida',
            'Purdue':'Purdue University',
            'Rutgers':'Rutgers, The State University of New Jersey',
            'Michigan State University':'Michigan State University',
            'University of Wisconsin':'University of Wisconsin System',
            'Indiana University':'Trustees of Indiana University',
            'Texas A & M':'Texas A & M University, College Station',
            'UC Irvine':'Regents of the University of California, Irvine',
            'UNC':'University of North Carolina at Chapel Hill', 
            'UVa':'University of Virginia', 
            'University of Minnesota':'University of Minnesota', 
            'Berkeley':'University of California, Berkeley',
            'University of Colorado':'Regents of the University of Colorado, Boulder',
            'University of Texas':'University of Texas, Austin',
            'UCSD':'Regents of the University of California, San Diego',
            'University of Washington':'University of Washington',
            'UC Davis':'Regents of the University of California, Davis',
            'University of Utah':'University of Utah',
            'UC Santa Barbara':'Regents of the University of California, Santa Barbara',
            'SUNY Stony Brook':'SUNY Research Foundation, Stony Brook',
            'University of Michigan':'Regents of the University of Michigan',
            'UC Santa Cruz':'Regents of the University of California, Santa Cruz',
            'University of Iowa':'University of Iowa',
            'SUNY Buffalo':'SUNY Research Foundation, Buffalo State College',
            'University of Arizona':'Arizona Board of Regents',
            'The Ohio State University':'Ohio State University',
            'University of Pittsburgh':'University of Pittsburgh',
            'University of Oregon':'University of Oregon',}

In [ ]:
'''
Let's loop through the AAU dictionary to get 
individual histories.
'''
aau_frames = {}
for key,value in aau_dict.items():
    institution = key
    df = aau_research[aau_research['Institution'] == value]
    history = df.groupby('YearAwarded')['AwardOutright'].sum().to_frame()
    aau_frames.update({institution:history})

In [ ]:
'''
Loop through the aau_frames and make lines graphs
comparing OU to individual AAU institutions.
'''
for key,value in aau_frames.items():
    merged = pd.merge(OU,value,on='YearAwarded')
    plot = merged.rename(columns={'AwardOutright_x':'OU','AwardOutright_y':key}).plot.line(color={'OU':'#841617',key:'#168483'},figsize=(10,5),title=('OU vs. ' + str(key)),xlabel='Year Awarded',ylabel='Amount ($)')
    fig = plot.get_figure()
    fig.savefig('../plots/OU-' + str(key) + '.png')
    print(plot)
    